# LocatNet

#### 0. Libraries

In [15]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from random import randrange
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#### 1. Data import and manipulation

<ul>
  <li>image_size - desired feature dimensions.</li>
  <li>EPOCHS - number of epochs when training the CNN.</li>
  <li>MAX_IMAGES - amount of image samples.</li>
</ul>

In [16]:
image_size = (200, 200)
EPOCHS = 50
MAX_IMAGES = 8000
IMAGE_COUNT = 0                     # is set later

Preprocess data to feature and label space.

In [17]:
def load_images_and_labels(image_folder, country_counts, labels_df, max_images = IMAGE_COUNT):
    images = []
    labels = []

    excluded_images_count = 0

    for index, row in tqdm(labels_df.iterrows(), total=min(len(labels_df), max_images), desc="Loading images"):
        if len(images) >= max_images:
            break

        currentCountry = row['country']
        count = country_counts.loc[country_counts['country'] == currentCountry].values[0]

        if count[1] > 19:                           # make sure there are at least 20 samples of every label
            img_path = os.path.join(image_folder, str(row['ID']) + '.png')

            img = cv2.imread(img_path)                  # load current image
            img = cv2.resize(img, image_size)           # resize
            img = img / 255.0                           # normalization

            images.append(img)
            labels.append(row['country'])
        else:                                       # skip low sample count labels
            excluded_images_count += 1
    
    print(f'{excluded_images_count} images were excluded due to low sample number.')
    print(f'{len(images)} images were loaded.')
    return np.array(images), np.array(labels)       # sample + label array

Retrieve the data using the function.

In [18]:
image_folder = 'Data'
labels_df = pd.read_csv('coordinates_with_country.csv')
country_counts = pd.read_csv('country_counts.csv')

IMAGE_COUNT = labels_df.shape[0]                        # set the default image count

In [ ]:
X, y = load_images_and_labels(image_folder, country_counts, labels_df, MAX_IMAGES)

In [20]:
def convert_to_uint8(image):
    # If the image is in float64 (CV_64F), we need to normalize and convert it to uint8
    if image.dtype == np.float64:
        image = (255 * (image - np.min(image)) / (np.max(image) - np.min(image))).astype(np.uint8)
    return image

In [ ]:
random_indices = random.sample(range(X.shape[0]), 5)

fig, axs = plt.subplots(1, 5, figsize=(15, 5))

# Iterate over the selected indices and plot each image
for i, idx in enumerate(random_indices):
    img = X[idx]
    img = convert_to_uint8(img)
    
    # Convert image from BGR (OpenCV default) to RGB for correct color display in matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Display the image
    axs[i].imshow(img_rgb)
    axs[i].axis('off')  # Hide the axes

    axs[i].set_title(f'{y[idx]}', fontsize=12)

plt.show()

Encode the labels <span style="color:green"><b>(country names)</b></span> into numbers.

In [22]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

Check dimensions of the data.

In [ ]:
print(f"Number of samples: {X.shape[0]}")
print(f"Dimension of image samples: {X.shape[1:3]}")
print(f"Color channels in the image samples: {X.shape[3]}")

Train-Test split.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, shuffle=True)

#### 2. Definition of the CNN

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # layers.Conv2D(128, (3, 3), activation='relu'),
    # layers.BatchNormalization(),
    # layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(np.unique(y_encoded)), activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

In [ ]:
model.summary()

#### 3. Data augmentation <font color='#66FF07'><b>(Optional)</b></font>

In [27]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the augmentation to your data
datagen.fit(X_train)

#### 4. Training and Evaluation

Fitting the model

##### <font color='red'><b>WITHOUT</b></font> data augmentation

In [ ]:
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=EPOCHS, validation_data=(X_test, y_test))

##### <font color='#66FF07'><b>WITH</b></font> data augmentation

In [ ]:
# Use the augmented data during training
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=EPOCHS, validation_data=(X_test, y_test))

#### 5. Testing

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Accuracy and Loss to Epochs graphs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Saving the model

In [ ]:
model.save('locatNet.h5')